Deep Learning Models -- A collection of various deep learning architectures, models, and tips for TensorFlow and PyTorch in Jupyter Notebooks.
- Author: Sebastian Raschka
- GitHub Repository: https://github.com/rasbt/deeplearning-models

In [1]:
%load_ext watermark
%watermark -a 'Sebastian Raschka' -v -p torch

Sebastian Raschka 

CPython 3.7.3
IPython 7.9.0

torch 1.7.0


- Runs on CPU or GPU (if available)

# Model Zoo -- Reproducible Results with Deterministic Behavior and Runtime Benchmark

In this notebook, we are benchmarking the performance impact of setting PyTorch to deterministic behavior. In general, there are two aspects for reproducible resuls in PyTorch, 
1. Setting a random seed
2. Setting cuDNN and PyTorch algorithmic behavior to deterministic

For more details, please see https://pytorch.org/docs/stable/notes/randomness.html

### 1. Setting a random seed

I recommend using a function like the following one prior to using dataset loaders and initializing a model if you want to ensure the data is shuffled in the same manner if you rerun this notebook and the model gets the same initial random weights:

In [2]:
def set_all_seeds(seed):
    os.environ["PL_GLOBAL_SEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

### 2. Setting cuDNN and PyTorch algorithmic behavior to deterministic

Similar to the `set_all_seeds` function above, I recommend setting the behavior of PyTorch and cuDNN to deterministic (this is particulary relevant when using GPUs). We can also define a function for that:

In [3]:
def set_deterministic():
    if torch.cuda.is_available():
        torch.backends.cudnn.benchmark = False
        torch.backends.cudnn.deterministic = True
    torch.set_deterministic(True)

# 1) Setup

After setting up the general configuration in this section, the following two sections will train a ResNet-101 model without and with deterministic behavior to get a sense how using deterministic options affect the runtime speed.

In [4]:
import os
import numpy as np
import torch
import random

In [5]:
##########################
### SETTINGS
##########################

# Device
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Device:', DEVICE)

# Data settings
num_classes = 10

# Hyperparameters
random_seed = 1
learning_rate = 0.01
batch_size = 128
num_epochs = 50

Device: cuda:0


# 2) Run without Deterministic Behavior

Before we enable deterministic behavior, we will run a ResNet-101 with otherwise the exact same settings for comparison. Note that setting random seeds doesn't affect the timing results.

In [6]:
### Set random seed ###
set_all_seeds(random_seed)

In [7]:
##########################
### Dataset
##########################

from deterministic_benchmark_utils import get_dataloaders


train_loader, test_loader = get_dataloaders(batch_size)

Files already downloaded and verified


In [8]:
##########################
### Model
##########################


from deterministic_benchmark_utils import resnet101




model = resnet101(num_classes, grayscale=False)

model = model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [9]:
from deterministic_benchmark_utils import train

train(num_epochs, train_loader, model, optimizer, DEVICE)

Epoch: 001/050 | Batch 0000/0391 | Loss: 2.8256
Epoch: 001/050 | Batch 0200/0391 | Loss: 3.0516
Epoch: 001/050 | Train: 25.898% |  Loss: 2.928
Time elapsed: 1.27 min
Epoch: 002/050 | Batch 0000/0391 | Loss: 1.8602
Epoch: 002/050 | Batch 0200/0391 | Loss: 1.7631
Epoch: 002/050 | Train: 18.342% |  Loss: 2.633
Time elapsed: 2.60 min
Epoch: 003/050 | Batch 0000/0391 | Loss: 1.5923
Epoch: 003/050 | Batch 0200/0391 | Loss: 1.4562
Epoch: 003/050 | Train: 43.756% |  Loss: 1.613
Time elapsed: 3.92 min
Epoch: 004/050 | Batch 0000/0391 | Loss: 1.3303
Epoch: 004/050 | Batch 0200/0391 | Loss: 1.4450
Epoch: 004/050 | Train: 55.988% |  Loss: 1.224
Time elapsed: 5.23 min
Epoch: 005/050 | Batch 0000/0391 | Loss: 1.2052
Epoch: 005/050 | Batch 0200/0391 | Loss: 1.2398
Epoch: 005/050 | Train: 59.410% |  Loss: 1.172
Time elapsed: 6.55 min
Epoch: 006/050 | Batch 0000/0391 | Loss: 1.2383
Epoch: 006/050 | Batch 0200/0391 | Loss: 1.0209
Epoch: 006/050 | Train: 64.646% |  Loss: 1.032
Time elapsed: 7.88 min
Epoc

# 3) Run with Deterministic Behavior

In this section, we set the deterministic behavior via the `set_deterministic()` function defined at the top of this notebook and compare how it affects the runtime speed of the ResNet-101 model. (Note that setting random seeds doesn't affect the timing results.)

In [10]:
set_deterministic()

In [11]:
### Set random seed ###
set_all_seeds(random_seed)

In [12]:
##########################
### Dataset
##########################

from deterministic_benchmark_utils import get_dataloaders


train_loader, test_loader = get_dataloaders(batch_size)

Files already downloaded and verified


In [13]:
##########################
### Model
##########################


from deterministic_benchmark_utils import resnet101




model = resnet101(num_classes, grayscale=False)

model = model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [14]:
from deterministic_benchmark_utils import train

train(num_epochs, train_loader, model, optimizer, DEVICE)

Epoch: 001/050 | Batch 0000/0391 | Loss: 2.8256
Epoch: 001/050 | Batch 0200/0391 | Loss: 2.4138
Epoch: 001/050 | Train: 24.776% |  Loss: 1.998
Time elapsed: 1.32 min
Epoch: 002/050 | Batch 0000/0391 | Loss: 1.8754
Epoch: 002/050 | Batch 0200/0391 | Loss: 1.7266
Epoch: 002/050 | Train: 32.594% |  Loss: 1.825
Time elapsed: 2.64 min
Epoch: 003/050 | Batch 0000/0391 | Loss: 1.5575
Epoch: 003/050 | Batch 0200/0391 | Loss: 1.4953
Epoch: 003/050 | Train: 43.622% |  Loss: 1.650
Time elapsed: 3.95 min
Epoch: 004/050 | Batch 0000/0391 | Loss: 1.4275
Epoch: 004/050 | Batch 0200/0391 | Loss: 1.5480
Epoch: 004/050 | Train: 46.606% |  Loss: 1.561
Time elapsed: 5.28 min
Epoch: 005/050 | Batch 0000/0391 | Loss: 1.3144
Epoch: 005/050 | Batch 0200/0391 | Loss: 1.8645
Epoch: 005/050 | Train: 36.120% |  Loss: 1.690
Time elapsed: 6.60 min
Epoch: 006/050 | Batch 0000/0391 | Loss: 1.7935
Epoch: 006/050 | Batch 0200/0391 | Loss: 1.5850
Epoch: 006/050 | Train: 44.868% |  Loss: 1.489
Time elapsed: 7.92 min
Epoc

# 4) Result

In this particular case, 66 min vs 65.92, the deterministic behavior does not seem to influence performance.